# Pruning

In [1]:
import torch
import copy
from indiv_utils import load_yaml, size_on_disk, get_layers, print_params_layer, measure_inference_latency, param_count, FLOPs_count, save_model_weights, start_train
from models import FFNN
from data_processing import MNISTDataProcessor

## Preliminaries & Setup

| hyperparameter  | MNIST |
| --------------- | ----- |
| learning rate   | 0.001 |
| batch size      | 64    |
| hidden size     | 1024  |
| # hidden layers | 2     |
| input size      | 20x20 |
| output size     | 10    |

In [2]:
"""Setup"""
# hyperparameters
lr = 0.001
batch_size = 64
num_hidden = 2
hidden_dim = 1024
out_dim = 10 # 10 MNIST classes   
epochs = 2
input_dim = 20*20

# config
config = load_yaml('config')

# device 
device = torch.device(config['device'])

# criterion
criterion = torch.nn.CrossEntropyLoss()

# model
model = FFNN(input_dim=input_dim, hidden_dim=hidden_dim, out_dim=out_dim, num_hidden=num_hidden, bias=True).to(device)

In [3]:
"""Model initialization:
Before training, SAVE the model's initial (random) weights. 
You will use them later for iterative pruning."""
initial_random_weights = save_model_weights(model, fname="initial")

Saved FFNN's weights as out/FFNN_weights_initial.pth.


In [4]:
"""Train"""
start_train(model, device, criterion, epochs, batch_size, lr)
trained_weights = save_model_weights(model, fname="trained")

Center Cropping images from 28x28 to 20x20
new image size:  (400,)
Center Cropping images from 28x28 to 20x20
new image size:  (400,)
parsing train features...
parsing test features...
The number of train labels: 60000
The number of test labels: 10000


938it [00:03, 276.29it/s]


Validating...
Epoch: 1/2, Step: 938/938, Train Accuracy: 97.196667%, Val Accuracy: 96.440%


938it [00:03, 289.96it/s]


Validating...
Epoch: 2/2, Step: 938/938, Train Accuracy: 98.275000%, Val Accuracy: 97.190%
Saved FFNN's weights as out/FFNN_weights_trained.pth.


In [5]:
"""load initial model"""
model_initial = FFNN(input_dim=input_dim, hidden_dim=hidden_dim, out_dim=out_dim, num_hidden=num_hidden, bias=True).to(device)
model_initial.load_state_dict(torch.load("out/FFNN_weights_initial.pth"))
print("Loaded initial model weights")

"""load trained model"""
model_trained = copy.deepcopy(model_initial)
model_trained.load_state_dict(torch.load("out/FFNN_weights_trained.pth"))
print("Loaded trained model weights")

"""test dataset"""
test_dataset = MNISTDataProcessor().vision_test_dataset()

Loaded initial model weights
Loaded trained model weights
Center Cropping images from 28x28 to 20x20
new image size:  (400,)
Center Cropping images from 28x28 to 20x20
new image size:  (400,)
parsing test features...
The number of test labels: 10000


In [6]:
"""Inference Latency of Trained Model"""
# Inference Latency
measure_inference_latency(model=model, test_dataset=test_dataset, device=device, warmup_itr=100)

Measuring inference latency of trained FFNN on cuda...


293it [00:00, 2928.57it/s]

Warm-up begins...


10000it [00:02, 3691.81it/s]


Mean inference latency: 0.120ms


In [10]:
"""Parameter Count, FLOPs, and Disk Storage of Trained Model"""
# layers
layers = get_layers(model=model)

# Parameter Count
param_count(model=model, layers=layers)

# FLOPs
FLOPs_count(model=model, layers=layers)

# Disk Storage
size_on_disk(model=model)

The number of parameters in each layer of FFNN:
	Linear: 400 * 1024 + 1024 = 410,624
	ReLU: = 0
	Linear: 1024 * 1024 + 1024 = 1,049,600
	ReLU: = 0
	Linear: 1024 * 10 + 10 = 10,250
The total number of parameters in FFNN: 1,470,474
The total FLOPs in FFNN: 0.002937 GFLOPs
Model Size on Disk: 5.883903 MB


5883903

## Magnitude pruning on SST2/MNIST

### Unstructured Magnitude (L1) Pruning 

### Iterative magnitude pruning (IMP)